# Introduction

This notebook implements a numerical solution of Problem 1 of the SIAM 100 digit challenge. [This particular solution was detailed by Walter Gautschi in a short paper titled "The numerical evaluation of a challenging integral"](http://www-m3.ma.tum.de/m3old/bornemann/challengebook/Chapter1/chall_int.pdf). Gautschi uses MATLAB to implement the algorithm and eventually reaches a precision of the solution to 64 digits.

In this Notebook, the solution uses the NumPy, SciPy and Numba Python libraries to arrive at a solution accurate to 13 digits. The focus of this work is to create a program that arrives at a reasonably accurate solution quickly. 

## Setup Notebook

In [1]:
import numpy as np
import scipy
from scipy.integrate import quad
import numba
from numba import njit

In [2]:
scipy.version.full_version

'1.6.3'

In [3]:
np.version.full_version

'1.20.3'

In [4]:
numba.__version__

'0.53.1'

# Lambert W Function

A central piece of the implementation described by Gautschi is evaluating the function $u(x)$. In it appears the [Lambert W function](https://en.wikipedia.org/wiki/Lambert_W_function).

SciPy provides an implementation of the Lambert W function; however, this section will show that we can get a performance increase by having our own implementation based on the Newton-Raphson method, as described in Section 4 of Gautschi's paper.

First, let's mesure the speed of the function $u$ when using the SciPy implementation of the Lambert W function:

In [5]:
from scipy.special import lambertw as W


def u_lambert(x):
    """
    Implementation of $u$ using the SciPy Lambert W function.
    """
    return np.exp(W(x))


%timeit u_lambert(1)

2.74 µs ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Now we implement $u(x)$ through the Newton-Raphson method as described by Gautschi and measure its performance. As the following code only uses Python and NumPy, it allows us to use Numba's JIT compiler to get an performance boost, something that is not normally possible when using a SciPy function (note, there is now a [new Numba project](https://github.com/numba/numba-scipy) which may change this).

In [6]:
@njit
def u_newton(x, tol=1e-11):
    """
    Implementation of $u$ using the Newton-Raphson method.
    """
    # initialise u making sure it is at least
    # positive as it will be used in a log method later
    u = max([u0(x), 1e-10]) 
    k = 0 # iteration counter
    while np.abs(u * np.log(u) - x) > tol and k < 1000:
        u = next_u(u, x)
        k += 1
    return u


@njit
def next_u(u, x):
    """
    Take a step towards the solution.
    """
    return (u + x)/(1 + np.log(u))


@njit
def u0(x):
    """
    Get an reasonably close estimate of the value as
    a first guess to save time in finding the solution.
    """
    return (
        1.0125 \
        + 0.857700000 * x \
        + 0.129013000 * x**2 \
        + 0.020864500 * x**3 \
        - 0.001761480 * x**4 \
        + 0.000057941 * x**5
    )


# run the method once to prompt it to compile
u_newton(1.0)


%timeit u_newton(1.0)

423 ns ± 61.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The SciPy implementation appears to take about 2.8 $\mu s$ whereas our own implementation takes about 50 $\mu s$. While there is a significant difference here in speed, it pays off
when we use this method embedded in the rest of the code!

# Full Solution Implementation

This implementation uses two more key features in addition to the Raphson-Newton method:
* SciPy's wrapper to the QUADPACK numerical integration tool
* A native implementation of the Epsilon Algorithm

The numerical integration function from SciPy is already a compiled program and well built, so it makes sense to use that implementation. The Espilon Algorithm is in contrast, straightforward to implement. By using the Python `map` function, this algorithm can quickly calculate the inital values for the algorithm.

Again, the inner functions in the integrals are simple functions of just numpy and native python components, so Numba can easily accelerate these.

In [7]:
@njit
def f1(x):
    """
    Inner function of the first integral
    """
    return np.cos(x)/(x + u_newton(x))


@njit
def f2(t, k):
    """
    Inner function of the second integral
    """
    return np.cos(t)/(t  + k*np.pi + u_newton(t + k * np.pi))


def integrate(func, lb, ub):
    """
    Calls SciPy's quad method to calculate the value of the integral of the
    given function.
    """
    return quad(func, lb, ub)[0]


def term_k(k):
    """
    Calculate the k-th term of the sum
    """
    return (-1)**k * integrate(lambda t: f2(t, k), -np.pi/2, np.pi/2)


def I(k):
    """
    Evaluate the integral series with k terms
    """
    return (
        integrate(f1, 0, np.pi/2) \
        + sum(map(term_k,range(1, k)))
    )


def epsilon_algorithm(func, N):
    """
    Applies the epsilon algorithm to a series evaluated by the function func with N terms.
    """
    # create the result container
    E = np.zeros((N, N+1))

    # calculate values of the series
    s = list(map(I,range(1, N+1)))
    
    # add them to the table
    for k in range(N):
        E[k, 1] = s[k]

    # run the main body of the algorithm
    for k in range(2, N+1):
        for m in range(0, N+2-(k+1)):
            E[m, k] = E[m+1, k-2] + 1/(E[m+1, k-1] - E[m, k-1])
    
    # return the value of interest
    return E[0, -1]


epsilon_algorithm(I, 25)

0.32336743167775234

In [8]:
%timeit epsilon_algorithm(I, 25)

4.27 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
